Association rules are normally written like this: {Diapers} -> {Beer} which means that there is a strong relationship between customers that purchased diapers and also purchased beer in the same transaction.

In the above example, the {Diaper} is the **antecedent** and the {Beer} is the **consequent**. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv('Transaction_SKU_Test.csv')
df.head(10)

,Name,SKU,Quantity
0,#1002,7870,1
1,#1002,9150,1
2,#1003,2010,1
3,#1004,8519,1
4,19135641007,2750,1
5,19135641008,8509,1
6,19135641009,9139,1
7,19135641010,3509,1
8,19135641011,7699,1
9,19135641012,5009,1


In [2]:
pd.pivot_table(df, index = ['SKU'], aggfunc=np.sum).head()

,Quantity
SKU,
1000,32
1001,59
1009,40
1050,48
1051,68


In [3]:
df['SKU'] = df['SKU'].astype('str')

In [4]:
basket = (df.groupby(['Name', 'SKU'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Name'))

In [5]:
basket.head()

SKU,1000,1001,1009,1050,1051,1059,1100,1101,1109,1150,...,91670,9169,9170,9179,9180,9189,92063,92075,92076,9945
Name,,,,,,,,,,,,,,,,,,,,,
#1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.91568E+11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.9157E+11,1.0,3.0,2.0,1.0,5.0,0.0,8.0,10.0,0.0,6.0,...,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0


In [6]:
# Show a subset of columns
basket.iloc[:,[0,1,2,3,4,5,6,7]].head()

SKU,1000,1001,1009,1050,1051,1059,1100,1101
Name,,,,,,,,
#1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.91568E+11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1.9157E+11,1.0,3.0,2.0,1.0,5.0,0.0,8.0,10.0


In [7]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head()

SKU,1000,1001,1009,1050,1051,1059,1100,1101,1109,1150,...,91670,9169,9170,9179,9180,9189,92063,92075,92076,9945
Name,,,,,,,,,,,,,,,,,,,,,
#1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.91568E+11,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1.9157E+11,1,1,1,1,1,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,1


In [8]:
# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True)

In [13]:
frequent_itemsets.sort_values('support', ascending=True).tail()

,support,itemsets
183,0.055117,[7980]
16,0.055451,[1350]
164,0.058784,[7820]
154,0.063229,[7770]
31,0.138460,[2010]


In [10]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules

,antecedants,consequents,support,confidence,lift
0,(1250),(1050),0.014224,0.085937,17.984920
1,(1050),(1250),0.004778,0.255814,17.984920
2,(1051),(1250),0.005890,0.245283,17.244546
3,(1250),(1051),0.014224,0.101562,17.244546
4,(1051),(1860),0.005890,0.169811,12.128032
5,(1860),(1051),0.014002,0.071429,12.128032
6,(1350),(1100),0.055451,0.026052,3.211546
7,(1100),(1350),0.008112,0.178082,3.211546
8,(2010),(1100),0.138460,0.007223,0.890422
9,(1100),(2010),0.008112,0.123288,0.890422


In [16]:
writer = pd.ExcelWriter("Association Analysis Result.xlsx", engine='xlsxwriter')
rules.to_excel(writer, sheet_name="Sheet1")
writer.save()